In [139]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip

In [6]:
import os
with os.scandir('./dw-data') as entries:
    for entry in entries:
        print(entry.name,'=',os.stat(entry).st_size/1024/1024)

201606scripts_sample.csv.gz = 17.89880657196045
201701scripts_sample.csv.gz = 18.13092613220215
chem.csv.gz = 0.03035449981689453
practices.csv.gz = 0.31778812408447266


In [7]:
# load the 2017 data

scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz', compression = 'gzip')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [8]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz', compression = 'gzip',names=col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [9]:
chem = pd.read_csv('./dw-data/chem.csv.gz', compression = 'gzip')
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [10]:
d = scripts.describe()
d

,items,nic,act_cost,quantity
count,973193.000000,973193.000000,973193.000000,973193.000000
mean,9.133136,73.058915,67.986613,741.329835
std,29.204198,188.070257,174.401703,3665.426958
min,1.000000,0.000000,0.040000,0.000000
25%,1.000000,7.800000,7.330000,28.000000
50%,2.000000,22.640000,21.220000,100.000000
75%,6.000000,65.000000,60.670000,350.000000
max,2384.000000,16320.000000,15108.320000,577720.000000


In [11]:
summary_stats = [(i,tuple(np.append(np.array(d[i]['count']*d[i]['mean']),(d.loc[['mean','std','25%','50%','75%'],i].values)))
                    ) for i in d.columns]
summary_stats

[('items', (8888304.0, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)),
 ('nic',
  (71100424.84000002, 73.05891517920908, 188.070256906825, 7.8, 22.64, 65.0)),
 ('act_cost',
  (66164096.11999998,
   67.98661326170655,
   174.40170332301963,
   7.33,
   21.22,
   60.67)),
 ('quantity',
  (721457006.0, 741.3298348837282, 3665.426958467915, 28.0, 100.0, 350.0))]

In [12]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [13]:
grouping = scripts.groupby('bnf_name')['items'].sum()

In [14]:
most_common_item = [(grouping.idxmax(), grouping.max() )]
most_common_item

[('Omeprazole_Cap E/C 20mg', 218583)]

In [15]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [16]:
merged = pd.merge(scripts,practices,left_on='practice',right_on='code')
merged

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
5,N85639,0406000T0,Prochlpzine Mal_Tab 5mg,1,0.97,0.91,28,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
6,N85639,0407010F0,Co-Codamol_Cap 30mg/500mg,1,0.84,0.89,24,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
7,N85639,0407010F0,Zapain_Tab 30mg/500mg,1,3.03,2.82,100,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
8,N85639,0407010H0,Paracet_Oral Susp Paed 120mg/5ml,1,0.62,0.69,100,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
9,N85639,0501011P0,Phenoxymethylpenicillin Pot_Tab 250mg,2,5.94,5.72,160,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [41]:
tocut = merged.sort_values(['practice','post_code'])
#myf = tocut.reset_index().drop('index',1)
#mylist = list(myf.to_records(index=False).astype(tuple))
#myf.merge(merged,how='inner',on=['practice','post_code'])
tocut = tocut[['practice','bnf_name','items','post_code']]
plist = tocut['practice'].unique()
newdf = pd.DataFrame()

for i in plist:
    temp = tocut[tocut['practice']==i][ tocut[tocut['practice']==i]['post_code'] == tocut[tocut['practice']==i]['post_code'].unique()[0]]
    newdf = newdf.append(temp)


In [43]:
newdf.head()

,practice,bnf_name,items,post_code
88503,A81005,Maalox_Susp 195mg/220mg/5ml S/F,1,TS14 7DJ
88504,A81005,Mucogel_Susp 195mg/220mg/5ml S/F,1,TS14 7DJ
88505,A81005,Maalox Plus_Susp S/F,1,TS14 7DJ
88506,A81005,Alginate_Raft-Forming Oral Susp S/F,6,TS14 7DJ
88507,A81005,Sod Algin/Pot Bicarb_Susp S/F,6,TS14 7DJ


In [44]:
common = newdf.groupby(['post_code','bnf_name']).agg({'items':sum})
postcodesum = common.reset_index().groupby('post_code').agg({'items':sum})
postcodesum.head()

,items
post_code,
B11 4BW,22731
B12 9LP,23236
B18 7AL,20508
B21 9RY,31027
B23 6DJ,28011


In [45]:
postcodemax = common.reset_index().sort_values(['post_code','items','bnf_name'],ascending=[True,False,True]).groupby('post_code').first()
postcodemax.head()

,bnf_name,items
post_code,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706
B12 9LP,Paracet_Tab 500mg,558
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556
B21 9RY,Metformin HCl_Tab 500mg,1033
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599


In [46]:
ratiodf = postcodemax.merge(postcodesum,on='post_code',suffixes=(None,'_total'))
ratiodf['items_ratio']=ratiodf['items']/ratiodf['items_total']
ratiodf = ratiodf.iloc[0:100,[0,3]]
ratiodf.head()

,bnf_name,items_ratio
post_code,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
B12 9LP,Paracet_Tab 500mg,0.024014
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
B21 9RY,Metformin HCl_Tab 500mg,0.033294
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [47]:
items_by_region = list(ratiodf.to_records().astype(tuple))

In [48]:
items_by_region[:5]

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.031058906339360346),
 ('B12 9LP', 'Paracet_Tab 500mg', 0.024014460320192804),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.027111371172225472),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03329358300834757),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.021384456106529576)]

In [49]:
grader.score.dw__items_by_region(items_by_region)

Your score:  0.9300000000000006


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [17]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [18]:
anom = merged[['practice','bnf_code','bnf_name','items','post_code']]
anom.head()

,practice,bnf_code,bnf_name,items,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,CH44 5UF


In [19]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [110]:
print(len(chem), chem['CHEM SUB'].nunique(), chem['NAME'].nunique())

3487 3481 2965


In [135]:
newchem = chem.drop_duplicates('CHEM SUB')
print(len(newchem), newchem['CHEM SUB'].nunique(), newchem['NAME'].nunique())

3481 3481 2959


In [25]:
newmy = pd.merge(anom,chem,left_on='bnf_code',right_on='CHEM SUB').drop('CHEM SUB',1)
newmy.head()

,practice,bnf_code,bnf_name,items,post_code,NAME
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,CH44 5UF,Bisacodyl
1,N81013,0106020C0,Bisacodyl_Tab E/C 5mg,19,SK11 6JL,Bisacodyl
2,N81029,0106020C0,Bisacodyl_Tab E/C 5mg,50,SK11 6JL,Bisacodyl
3,N81029,0106020C0,Bisacodyl_Suppos 10mg,1,SK11 6JL,Bisacodyl
4,N81029,0106020C0,Dulcolax_Tab 5mg,1,SK11 6JL,Bisacodyl


In [116]:
newmy['opioids']=newmy['NAME'].str.lower().str.contains('|'.join(opioids)) 
newmy['opioids'].value_counts()

False    957276
True      40619
Name: opioids, dtype: int64

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [117]:
newmy.head()

,practice,bnf_code,bnf_name,items,post_code,NAME,opioids
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,CH44 5UF,Bisacodyl,False
1,N81013,0106020C0,Bisacodyl_Tab E/C 5mg,19,SK11 6JL,Bisacodyl,False
2,N81029,0106020C0,Bisacodyl_Tab E/C 5mg,50,SK11 6JL,Bisacodyl,False
3,N81029,0106020C0,Bisacodyl_Suppos 10mg,1,SK11 6JL,Bisacodyl,False
4,N81029,0106020C0,Dulcolax_Tab 5mg,1,SK11 6JL,Bisacodyl,False


In [118]:
opioids_per_practice = newmy.groupby('practice')['opioids'].sum()
opioids_per_practice

practice
A81005    50.0
A81007    63.0
A81011    73.0
A81012    57.0
A81017    82.0
          ... 
Y05570     2.0
Y05583     0.0
Y05597     0.0
Y05660     8.0
Y05670     0.0
Name: opioids, Length: 850, dtype: float64

In [119]:
pres_per_practice = newmy.groupby('practice').bnf_name.agg('count')
pres_per_practice

practice
A81005    1268
A81007    1219
A81011    1357
A81012    1173
A81017    1843
          ... 
Y05570      22
Y05583      28
Y05597      18
Y05660     314
Y05670      11
Name: bnf_name, Length: 850, dtype: int64

In [120]:
ratio_per_practice = opioids_per_practice/pres_per_practice
ratio_per_practice

practice
A81005    0.039432
A81007    0.051682
A81011    0.053795
A81012    0.048593
A81017    0.044493
            ...   
Y05570    0.090909
Y05583    0.000000
Y05597    0.000000
Y05660    0.025478
Y05670    0.000000
Length: 850, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [121]:
totalopioids = np.mean(newmy['opioids'])
totalopioids

0.04070468335846958

In [122]:
relative_opioids_per_practice = ratio_per_practice - totalopioids
relative_opioids_per_practice

practice
A81005   -0.001273
A81007    0.010977
A81011    0.013090
A81012    0.007889
A81017    0.003788
            ...   
Y05570    0.050204
Y05583   -0.040705
Y05597   -0.040705
Y05660   -0.015227
Y05670   -0.040705
Length: 850, dtype: float64

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [123]:
np.std(ratio_per_practice)
np.std(newmy['opioids'])

0.1976051925207337

In [124]:
standard_error_per_practice = np.std(newmy['opioids']) / np.sqrt(pres_per_practice)
standard_error_per_practice

practice
A81005    0.005549
A81007    0.005660
A81011    0.005364
A81012    0.005770
A81017    0.004603
            ...   
Y05570    0.042130
Y05583    0.037344
Y05597    0.046576
Y05660    0.011152
Y05670    0.059580
Name: bnf_name, Length: 850, dtype: float64

In [125]:
opioid_scores = relative_opioids_per_practice / standard_error_per_practice
opioid_scores

practice
A81005   -0.229309
A81007    1.939492
A81011    2.440317
A81012    1.367270
A81017    0.822950
            ...   
Y05570    1.191667
Y05583   -1.089996
Y05597   -0.873941
Y05660   -1.365464
Y05670   -0.683191
Length: 850, dtype: float64

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [126]:
opzscore = opioid_scores.sort_values(ascending=False)[:100]
opzscore

practice
Y01852    10.931354
Y03006     6.865449
Y03668     5.740604
P81051     4.949748
G81703     4.668089
            ...    
A81064     1.862710
A88011     1.824404
N81096     1.817166
D81043     1.812729
M91612     1.803660
Length: 100, dtype: float64

In [127]:
pname = practices.set_index('code')
pname = pname['name']
pname = pname.loc[opzscore.index]
pnamedf = pd.DataFrame(pname).groupby('code').first()
pnamedf.head()

,name
code,
A81007,BANKHOUSE SURGERY
A81011,CHADWICK PRACTICE
A81058,COULBY MEDICAL PRACTICE
A81064,THE DISCOVERY PRACTICE
A81610,THE ROSEBERRY PRACTICE


In [128]:
opzscoredf = pd.DataFrame(opzscore).sort_index()
opzscoredf.head()

,0
practice,
A81007,1.939492
A81011,2.440317
A81058,1.895320
A81064,1.862710
A81610,2.907737


In [129]:
pres_per_practicedf = pd.DataFrame(pres_per_practice).sort_index()
pres_per_practicedf = pres_per_practicedf.loc[pnamedf.index]
pres_per_practicedf.head()

,bnf_name
code,
A81007,1219
A81011,1357
A81058,1400
A81064,1122
A81610,1287


In [130]:
zscoretupledf = pd.concat([pnamedf,opzscoredf,pres_per_practicedf],axis=1,join='inner').sort_values(0,ascending=False)
zscoretupledf.head()

,name,0,bnf_name
Y01852,NATIONAL ENHANCED SERVICE,10.931354,7
Y03006,OUTREACH SERVICE NH / RH,6.865449,2
Y03668,BRISDOC HEALTHCARE SERVICES OOH,5.740604,58
P81051,DARWEN HEALTHCARE,4.949748,1671
G81703,H&R P C SPECIAL SCHEME,4.668089,36


In [131]:
anomalies = list(zscoretupledf.to_records(index=False).astype(tuple))
anomalies[:5]

[('NATIONAL ENHANCED SERVICE', 10.931354288509278, 7),
 ('OUTREACH SERVICE NH / RH', 6.8654493832347, 2),
 ('BRISDOC HEALTHCARE SERVICES OOH', 5.740603848422584, 58),
 ('DARWEN HEALTHCARE', 4.949747982094348, 1671),
 ('H&R P C SPECIAL SCHEME', 4.668088701257496, 36)]

In [ ]:
unique_practices = ...
anomaliesmy = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [ ]:
results.head()

In [132]:
grader.score.dw__script_anomalies(anomalies)

Your score:  0.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [ ]:
scripts16 = ...

In [ ]:
script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [ ]:
grader.score.dw__script_growth(script_growth)

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [ ]:
p = ...
rates = ...
rare_codes = ...
scripts['rare'] = ...

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
rare_cost_prop = ...

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = ...

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
standard_errors = ...

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
rare_scores = ...

In [ ]:
rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [ ]:
grader.score.dw__rare_scripts(rare_scripts)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*